### Trying to solve Task 1A now!

In [340]:
import pandas as pd
import numpy as np

splits = 10
trainUrl = 'https://raw.githubusercontent.com/Toroto006/iml2021/main/task1A/train.csv?token=AH44KNS2D2RZ2TH7Z45HDQLAKDGKG'

trainPD = pd.read_csv(trainUrl)
#trainPD

We now have the data as pd objects and can start working with it.

Maybe doing the 10-fold split already makes sense:

In [341]:
# Just some static splitSize fold splits
if trainPD.size % splits != 0:
  print(f"ERROR! Not a possible splits amount, because %i is over" % (trainPD.size % splits))
splitSize = int(trainPD.size / splits)

def staticSplit():
  isplit = []
  for i in range(splitSize):
    isplit.append(trainPD.loc[i*15:i*15+14])
  return isplit

def randShuffleSplit():
  # First sample (i.e. deepcopy & shuffel the data)
  randTrainPD = trainPD.sample(frac=1).reset_index(drop=True)
  #then do the normal calculations again
  isplit = []
  for i in range(splitSize):
    isplit.append(randTrainPD.loc[i*15:i*15+14])
  return isplit

#split = randShuffleSplit()
split = staticSplit()
#split[9]

As we need train data and test data for the cross validation, let's build that from our splits here:

In [342]:
# Position i contains the i'th train and i'th test data.
train = {}
test = split
for i in range(0, splits):
  for j in range(0, splits):
    if i != j:
      if len(train) == i:
        train[i] = split[j]
      else:
        train[i] = train[i].append(split[j])

#train[0]

In [343]:
# are given to us
lambdas = [0.1, 1, 10, 100, 200]

This shoud return the Root Mean Squared Error

In [344]:
def rmse(predictions, targets):
  # Should diretly work on pd (hopefully)
  return np.sqrt(np.mean((predictions-targets)**2))

This should be the training of the ridgeRegression.
Hence it should return the trained matrix, afterwards used by the predictRR to 
predict y's.

In [345]:
def ridgeRegression(train, alp):
  #np.linalg.norm is the L2 norm for ||w||_2 use .square to square it afterwards
  # to do ||w||_2^2 is just np.dot() ???

  # lets first try with sklearn
  from sklearn import linear_model
  # alpha is our lambda
  model = linear_model.Ridge(alpha=alp)
  model.fit(train.loc[:,'x1':'x13'], train['y'])
  return model
#r = ridgeRegression(train[0], lambdas[0])

This is the predictRR, which gets a dataFrame of which it only uses x1 to x13 and it's given trained matrix to predict the y's.

In [346]:
def predictRR(trained, test:pd.DataFrame):
  #use w_hat = trained we get to calculate
  #test = test['x1':'x13']

  # lets first try with sklearn
  return trained.predict(test.loc[:,'x1':'x13'])

#predictRR(r, test[0])

Now let's do the evaluation for our test results.

In [347]:
avgRMSE = []
for lam in lambdas:
  lams = []
  for i in range(0, splits):
    # trained is w
    trained = ridgeRegression(train[i], lam)
    # tests contains the predicted y
    tests = predictRR(trained, test[i])
    lamRmse = rmse(tests, test[i]['y'])
    lams.append(lamRmse)
  avg = np.average(lams)
  avgRMSE.append(avg)

#print(avgRMSE)

Output the data for submission.

In [348]:
res = pd.DataFrame(data=avgRMSE)
res.to_csv("testOut.csv", index=False, header=False)
res

,0
0,5.501809
1,5.499839
2,5.483631
3,5.636642
4,5.721234
